In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
# Load the selected data into an initial data frame from the amazon data reviews

from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
initial_load_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Display the first 20 Records
initial_load_df.show()

In [4]:
# Check the initial_load dataframe count

initial_load_df.count()

1785997

In [ ]:
# Create the vine_table. DataFrame

vine_df = initial_load_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)

In [6]:
# Check the vine dataframe count

vine_df.count()

1785997

In [ ]:
# Filter the data and create a new DataFrame or table to retrieve all the rows where the total_votes count 
# is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

vine_df_filtered_GE20 = vine_df.where(vine_df.total_votes>= 20)
vine_df_filtered_GE20.show()



In [8]:
# Check the vine filtered dataframe count

vine_df_filtered_GE20.count()

65379

In [ ]:
# Filter the vine data frame futher by calculating the percentage of helpful_votes in realation to total votes , only keep where the percentage is equal to greater than 50%

vine_df_filtered_HV_TV_PER_GE_50 = vine_df_filtered_GE20.where((vine_df_filtered_GE20.helpful_votes/vine_df_filtered_GE20.total_votes) >= 0.5)
vine_df_filtered_HV_TV_PER_GE_50.show()

In [10]:
# Check the new filtered vine filtered dataframe count

vine_df_filtered_HV_TV_PER_GE_50.count()

40565

In [ ]:
# Create a dataframe from the filtered dataframe , that contains only paid vine program reviews (vine equals Y)

vine_df_filtered_vine_IS_Y = vine_df_filtered_HV_TV_PER_GE_50.where(vine_df_filtered_HV_TV_PER_GE_50.vine=='Y')
vine_df_filtered_vine_IS_Y.show()

In [22]:
# Check the number of entries in the dataframe

total_number_of_paid_reviews = vine_df_filtered_vine_IS_Y.count()

print(total_number_of_paid_reviews)

94


In [ ]:
# Create a dataframe from the filtered dataframe , that contains only paid vine program reviews (vine equals N)

vine_df_filtered_vine_IS_N = vine_df_filtered_HV_TV_PER_GE_50.where(vine_df_filtered_HV_TV_PER_GE_50.vine=='N')
vine_df_filtered_vine_IS_N.show()

In [23]:
# Check the number of entries in the dataframe

total_number_of_unpaid_reviews = vine_df_filtered_vine_IS_N.count()

print(total_number_of_unpaid_reviews)

40471


In [24]:
five_star_paid_reviews = vine_df_filtered_vine_IS_Y.select('review_id').where(vine_df_filtered_vine_IS_Y.star_rating==5).count()

print(five_star_paid_reviews)

48


In [25]:
five_star_unpaid_reviews = vine_df_filtered_vine_IS_N.where(vine_df_filtered_vine_IS_N.star_rating==5).count()
print(five_star_unpaid_reviews)

15663


In [ ]:
from pyspark.sql.functions import col,when,count

vine_df_filtered_vine_IS_N.groupby("star_rating").agg(
                                                  count(when(col("star_rating")==5,True)),
                                                  count(when(col("star_rating")<=4,True))).show()

In [ ]:
vine_df_filtered_vine_IS_Y.groupby("star_rating").agg(
                                                  count(when(col("star_rating")==5,True)),
                                                  count(when(col("star_rating")<=4,True))).show()

In [27]:
# Percentage of 5-Star paid reviews
(five_star_unpaid_reviews / total_number_of_unpaid_reviews) * 100


38.701786464381904

In [28]:
# Percentage of 5-Star paid reviews
(five_star_paid_reviews / total_number_of_paid_reviews) * 100


51.06382978723404